# TFRecor数据存储

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data",
                                  dtype=tf.uint8, one_hot=True)


# 定义生成整数型和字符串型属性的方法，这是将数据填入到Example协议内存块
# (protocol buffer)的第一步，以后会调用到这个方法
def Int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def Bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


# 读取mnist数据。
images = mnist.train.images

labels = mnist.train.labels

pixels = images.shape[1]
print(pixels)
num_examples = mnist.train.num_examples
print(num_examples)
#输出TFRecord文件的地址(相对于系统根目录)。
filename = "MNIST_tfrecords/MNIST_tfrecords"

# 创建一个python_io.TFRecordWriter()类的实例
writer = tf.python_io.TFRecordWriter(filename)

# for循环执行了将数据填入到Example协议内存块的主要操作
for i in range(num_examples):
    # 将图像矩阵转化成一个字符串
    image_to_string = images[i].tostring()

    feature = {
        "pixels": Int64_feature(pixels),
        "label": Int64_feature(np.argmax(labels[i])),
        "image_raw": Bytes_feature(image_to_string)
    }
    features = tf.train.Features(feature=feature)

    # 定义一个Example，将相关信息写入到这个数据结构
    example = tf.train.Example(features=features)

    # 将一个Example写入到TFRecord文件
    # 原型writer(self, record)
    writer.write(example.SerializeToString())

# 在写完文件后最好的习惯是调用close()函数关闭
writer.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
784
55000


# 读取TF数据

In [9]:
import tensorflow as tf


#创建一个TFRecordReader类的实例
reader = tf.TFRecordReader()

#创建一个队列对输入文件列表进行维护，队列的知识放到了本章的稍后
#函数原型string_input_producer(string_tensor,num_epochs,shuffle,seed,
#                                  capacity,shared_name,name,cancel_op)
filename_queue = tf.train.string_input_producer(
                       ["MNIST_tfrecords/MNIST_tfrecords"])

#使用TFRecordReader.read()函数从文件中读取一个样例，原型reader(self,queue,name)
#也可使用read_up_to()函数一次性读取多个样例，
#原型read_up_to(self,queue,num_records,name)
_,serialized_example = reader.read(filename_queue)

#使用parse_single_example()函数解析读取的样例。
#原型parse_single_example(serialized,features,name,example_names)
features = tf.parse_single_example(
    serialized_example,
    features={
        #可以使用FixedLenFeature类对属性进行解析，
        "image_raw":tf.FixedLenFeature([],tf.string),
        "pixels":tf.FixedLenFeature([],tf.int64),
        "label":tf.FixedLenFeature([],tf.int64)
    })

#decode_raw()函数用于将字符串解析成图像对应的像素数组
#函数原型decode_raw(bytes,out_type,little_endian,name)
images = tf.decode_raw(features["image_raw"],tf.uint8)
#使用cast()函数进行类型转换
labels = tf.cast(features["label"],tf.int32)
pixels = tf.cast(features["pixels"],tf.int32)


with tf.Session() as sess:
    #启动多线程处理输入数据，多线程处理数据也会在本章的稍后予以介绍
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coordinator)

    for i in range(10):
        image, label, pixel = sess.run([images, labels, pixels])
        print(label)
        #输出7 3 4 6 1 8 1 0 9 8



7
3
4
6
1
8
1
0
9
8


In [2]:
import  tensorflow as tf
import  numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",
                                             dtype=tf.uint8,one_hot=True)

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

images = mnist.test.images
labels = mnist.test.labels
pixels = images.shape[1]
num_examples = mnist.test.num_examples

#num_files定义总共写入多少个文件
num_files = 2

for i in range(num_files):
    #将数据写入多个文件时，为区分这些文件可以添加后缀
    filename = ("TFRecord/data_tfrecords-%.1d-of-%.1d"
                                                         % (i, num_files))
    writer = tf.python_io.TFRecordWriter(filename)

    #将Example结构写入TFRecord文件，写入文件的过程和11.1节一样。
    for index in range(num_examples):
        image_string = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            "pixels": _int64_feature(pixels),
            "label": _int64_feature(np.argmax(labels[index])),
            "image_raw": _bytes_feature(image_string)
        }))
        writer.write(example.SerializeToString())
    writer.close()

W1211 15:01:48.445559 140669260474176 deprecation.py:323] From <ipython-input-2-64bab3ec9496>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1211 15:01:48.446966 140669260474176 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1211 15:01:48.448385 140669260474176 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed 

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1211 15:01:48.741093 140669260474176 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1211 15:01:48.744842 140669260474176 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1211 15:01:48.797538 140669260474176 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
import  tensorflow as tf

#使用match_filenames_once()函数获取符合正则表达式的所有文件
#函数原型match_filenames_once(pattern,name)
files = tf.train.match_filenames_once("TFRecord/data_tfrecords-*")

#通过string_input_producer()函数创建输入队列，输入队列的文件列表是files
#函数原型string_input_producer(string_tensor,num_epochs,shuffle,capacity,
#                                                  shared_name,name,cancel_op)
#函数的shuffle参数用于指定是否随即打乱读文件的顺序，在实际问题中会设置为True
filename_queue = tf.train.string_input_producer(files, shuffle=False)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

#解析读取的样例。
features = tf.parse_single_example(
    serialized_example,
    features={
        "image_raw":tf.FixedLenFeature([],tf.string),
        "pixels":tf.FixedLenFeature([],tf.int64),
        "label":tf.FixedLenFeature([],tf.int64)
    })
images = tf.decode_raw(features["image_raw"],tf.uint8)
labels = tf.cast(features["label"],tf.int32)
pixels = tf.cast(features["pixels"],tf.int32)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    #print(sess.run(files))
    #打印文件列表，输出为：
    #[b'/home/jiangziyang/TFRecord/data_tfrecords-1-of-2'
    # b'/home/jiangziyang/TFRecord/data_tfrecords-0-of-2']

    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coordinator)
#     for i in range(6):
#         print(sess.run([images,labels]))

    coordinator.request_stop()
    coordinator.join(threads)

In [4]:
import tensorflow as tf

files = tf.train.match_filenames_once("TFRecord/data_tfrecords-*")
#创建文件队列shuffle参数设置为True，打乱文件
filename_queue = tf.train.string_input_producer(files, shuffle=True)

#实例化TFRecordReader类，准备读取TFRecord文件
reader = tf.TFRecordReader()
_,serialized_example = reader.read(filename_queue)

#解析读取的样例，这部分和11.1.1节是相同的
features = tf.parse_single_example(
    serialized_example,
    features={
        "image_raw":tf.FixedLenFeature([],tf.string),
        "pixels":tf.FixedLenFeature([],tf.int64),
        "label":tf.FixedLenFeature([],tf.int64)
    })
images = tf.decode_raw(features["image_raw"],tf.uint8)
labels = tf.cast(features["label"],tf.int32)
pixels = tf.cast(features["pixels"],tf.int32)

#设置每个batch中样例的个数
batch_size =50

#用于组合成batch的队列中最多可以缓存的样例的个数
capacity = 5000 + 3 * batch_size

#set_shape()函数用来设置设置尺寸，这一步操作也可以使用image.resize_images()
#函数来完成。设置尺寸的操作是必须的，在1.0.0版的TensorFlow中如果没有这一步
#会在batch()函数的capacity参数处报错ValueError: All shapes must be fully
#defined: [TensorShape([Dimension(None)]), TensorShape([])]
images.set_shape(784)

#使用batch()函数将样例组合成batch
#函数原型batch(tensors,batch_size,num_threads,capacity,enqueue_many,shapes,
#                        dynamic_pad,allow_smaller_final_batch,shared_name,name)
image_batch, label_batch = tf.train.batch([images, labels],
                                    batch_size=batch_size,capacity=capacity,)



#shuffle_batch()函数会在组织样例数据成batch之前将数据顺序打乱，
#这两个函数的使用情况类似。shuffle_batch()函数原型为
#shuffle_batch(tensors,batch_size,capacity,min_after_dequeue,num_threads,
#                seed,enqueue_many,shapes,allow_smaller_final_batch,shared_name,name)
#min_after_dequeue=5000
#image_batch, label_batch = tf.train.shuffle_batch([images, labels],
#                                    batch_size=batch_size,capacity=capacity,
#                                                  min_after_dequeue=min_after_dequeue)

#接下来就可以像之前的网络模型那样创建会话并开始训练了，与之前的训练过程相比，
#这里的会话中增加了多线程处理的相关代码
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    #一般在这个循环内开始训练，这里设定了训练轮数为3
    #在每一轮训练的过程中都会执行一个组合样例为batch的操作并打印出来
    for i in range(3):
        xs,ys=sess.run([image_batch,label_batch])
        print(xs,ys)

    coord.request_stop()
    coord.join(threads)

OutOfRangeError: FIFOQueue '_16_batch_2/fifo_queue' is closed and has insufficient elements (requested 50, current size 0)
	 [[node batch_2 (defined at <ipython-input-4-0e31e8bbd50c>:39) ]]

Original stack trace for 'batch_2':
  File "/home/liyuan3970/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/liyuan3970/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/liyuan3970/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/liyuan3970/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/liyuan3970/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/liyuan3970/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loo

In [ ]:
import tensorflow as tf

files = tf.train.match_filenames_once("TFRecord/data_tfrecords-*")
filename_queue = tf.train.string_input_producer(files, shuffle=True)

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

features = tf.parse_single_example(
    serialized_example,
    features={
        "image_raw": tf.FixedLenFeature([], tf.string),
        "pixels": tf.FixedLenFeature([], tf.int64),
        "label": tf.FixedLenFeature([], tf.int64)
    })
images = tf.decode_raw(features["image_raw"], tf.uint8)
labels = tf.cast(features["label"], tf.int32)
pixels = tf.cast(features["pixels"], tf.int32)

batch_size = 10

capacity = 5000 + 3 * batch_size

images.set_shape(784)

# 使用shuffle_batch()函数在组织样例数据成batch之前将样例数据顺序打乱，
# 对于min_after_dequeue参数，假设是100
min_after_dequeue = 100
image_batch, label_batch = tf.train.shuffle_batch([images, labels],
                                                  batch_size=batch_size, capacity=capacity,
                                                  min_after_dequeue=min_after_dequeue)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for i in range(3):
        xs, ys = sess.run([image_batch, label_batch])
        print(xs, ys)

    coord.request_stop()
    coord.join(threads)

'''打印的内容
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [3 9 4 1 9 6 9 3 5 4]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [9 4 3 7 0 1 1 3 7 0]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [6 1 7 3 1 4 5 8 3 7]
'''